## How to Grid Search Hyperparameters for Deep Learning Models in Python with Keras

- https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/

In [1]:
# Use scikit-learn to grid search the dropout rate
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.constraints import MaxNorm
from scikeras.wrappers import KerasClassifier

In [2]:
# Function to create model, required for KerasClassifier
def create_model(dropout_rate, weight_constraint):
    # create model
    model = Sequential()
    model.add(Dense(12, input_shape=(8,), kernel_initializer='uniform', activation='linear', kernel_constraint=MaxNorm(weight_constraint)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))
    
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    # return values
    return model

In [3]:
# fix random seed for reproducibility
tf.random.set_seed(seed = 7)

In [4]:
# load dataset
dataset = np.loadtxt("dataset/diabetes.csv", delimiter=",")

In [5]:
# split into input (X) and output (Y) variables
X = dataset[:,0:8]
Y = dataset[:,8]

In [6]:
# show to dataset
pd.concat([
    pd.DataFrame(X, columns=['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8']),
    pd.DataFrame(Y, columns=['Y'])
], axis=1)

x1     x2    x3    x4     x5    x6     x7    x8    Y
0     6.0  148.0  72.0  35.0    0.0  33.6  0.627  50.0  1.0
1     1.0   85.0  66.0  29.0    0.0  26.6  0.351  31.0  0.0
2     8.0  183.0  64.0   0.0    0.0  23.3  0.672  32.0  1.0
3     1.0   89.0  66.0  23.0   94.0  28.1  0.167  21.0  0.0
4     0.0  137.0  40.0  35.0  168.0  43.1  2.288  33.0  1.0
..    ...    ...   ...   ...    ...   ...    ...   ...  ...
763  10.0  101.0  76.0  48.0  180.0  32.9  0.171  63.0  0.0
764   2.0  122.0  70.0  27.0    0.0  36.8  0.340  27.0  0.0
765   5.0  121.0  72.0  23.0  112.0  26.2  0.245  30.0  0.0
766   1.0  126.0  60.0   0.0    0.0  30.1  0.349  47.0  1.0
767   1.0   93.0  70.0  31.0    0.0  30.4  0.315  23.0  0.0

[768 rows x 9 columns]

In [7]:
# create model
model = KerasClassifier(model=create_model, epochs=100, batch_size=10, verbose=0)

In [8]:
model

KerasClassifier(
	model=<function create_model at 0x000001761EA943A0>
	build_fn=None
	warm_start=False
	random_state=None
	optimizer=rmsprop
	loss=None
	metrics=None
	batch_size=10
	validation_batch_size=None
	verbose=0
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=100
	class_weight=None
)

In [9]:
# define the grid search parameters
weight_constraint = [1.0, 2.0, 3.0, 4.0, 5.0]
dropout_rate = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

In [10]:
param_grid = dict(model__dropout_rate=dropout_rate, model__weight_constraint=weight_constraint)

In [11]:
param_grid

{'model__dropout_rate': [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
 'model__weight_constraint': [1.0, 2.0, 3.0, 4.0, 5.0]}

In [12]:
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)

In [13]:
grid_result = grid.fit(X, Y)

In [14]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.730469 using {'model__dropout_rate': 0.0, 'model__weight_constraint': 2.0}
0.699219 (0.003189) with: {'model__dropout_rate': 0.0, 'model__weight_constraint': 1.0}
0.730469 (0.011500) with: {'model__dropout_rate': 0.0, 'model__weight_constraint': 2.0}
0.714844 (0.008438) with: {'model__dropout_rate': 0.0, 'model__weight_constraint': 3.0}
0.717448 (0.006639) with: {'model__dropout_rate': 0.0, 'model__weight_constraint': 4.0}
0.713542 (0.009744) with: {'model__dropout_rate': 0.0, 'model__weight_constraint': 5.0}
0.712240 (0.010253) with: {'model__dropout_rate': 0.1, 'model__weight_constraint': 1.0}
0.708333 (0.010253) with: {'model__dropout_rate': 0.1, 'model__weight_constraint': 2.0}
0.701823 (0.004872) with: {'model__dropout_rate': 0.1, 'model__weight_constraint': 3.0}
0.707031 (0.008438) with: {'model__dropout_rate': 0.1, 'model__weight_constraint': 4.0}
0.722656 (0.013902) with: {'model__dropout_rate': 0.1, 'model__weight_constraint': 5.0}
0.717448 (0.014731) with: {'model__dr